# Match BGBM Collectors to Wikidata Items Using *Nearest Neighbour*

Basically we attempt a match of `canonical_string` of WikiData to `canonical_string` of the collectors (in this case the names were parsed beforehand into single names using <https://libraries.io/rubygems/dwc_agent>)

TODO:

- review code: match when full collector name was given instead of an abbreviated collector name
- review code: evaluate if multiple names (WikiData or collector data) are found
- match also with time periode of work (WikiData) ⇌ created time of the herbarium sheet (if no other life time data are available)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
wikidata = pd.read_csv("data/wikidata_persons_botanists_20230703_1352.csv", index_col=0, low_memory=False)

wikidata.head()

,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768.0,1826.0,NaN,NaN,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818.0,1904.0,NaN,NaN,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718.0,1790.0,NaN,NaN,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805.0,1860.0,NaN,NaN,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808.0,1880.0,NaN,NaN,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# Create data frame with unique canonical strings 
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()

wd_matchtest

# colls = list(wikidata.columns)

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


In [3]:
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

wd_matchtest_fullnames


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


### Load Collectors Data Set

Data sources:

- option 1: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)
- option 2: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>, it can parse text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."` (see [./bin/README.md](bin/README.md))

TODO:
- check parsed fields `particle` and other fields, e.g. «`Abbas al Ani, H.`» to get a proper abbreviated atomized name

    ```bash
    cd data/VHde_0195853-230224095556074_BGBM/
    head occurrence_recordedBy_occurrenceIDs_20230524_parsed.tsv \
      | column --table --separator $'\t' \
      | sed 's@^@  # @;'
      # family     given  suffix  particle  dropping_particle  nick  appellation  title  occurrenceID_count  occurrenceID_first
      #            No                                                                    1                   http://id.snsb.info/snsb/collection/108286/167064/109352
      # Azofeifa   A.                                                                    2                   https://herbarium.bgbm.org/object/B200211416
      # A. Cano    E.                                                                    1                   https://herbarium.bgbm.org/object/B100699397
      # Henry      A.                                                                    1                   https://herbarium.bgbm.org/object/B200098813
      # Selmons    Ad                                                                    1                   https://herbarium.bgbm.org/object/B100379213
      # Aaronsohn  A.                                                                    3                   https://je.jacq.org/JE00010154
      ```


In [4]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_0195853-230224095556074_BGBM/occurrence_recordedBy_occurrenceIDs_20230524_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.
collectors.sort_values(by=['family', 'given','occurrenceID_first'], inplace=True)
collectors

,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first
2,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100699397
39762,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B101149305
5,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://je.jacq.org/JE00010154
26985,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://herbarium.bgbm.org/object/B100217620
26989,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,2,https://herbarium.bgbm.org/object/B100326682
...,...,...,...,...,...,...,...,...,...,...
66575,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,5,https://herbarium.bgbm.org/object/B100263330
32851,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100263331
66576,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://herbarium.bgbm.org/object/B100009590
66577,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100530714


#### Check Composition of Parsed Collector Data

In [5]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (1395 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first
2213,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B200125981
27076,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100184021
21037,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,1,http://id.snsb.info/snsb/collection/22086/3086...
18465,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100296455
361,Aghababyan,Mvon,NaN,NaN,NaN,NaN,NaN,NaN,96,https://herbarium.bgbm.org/object/B100576238
...,...,...,...,...,...,...,...,...,...,...
65787,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,3,https://je.jacq.org/JE04006629
65788,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,1,https://je.jacq.org/JE04007139
65874,Ziz,Johann Baptist,NaN,NaN,NaN,NaN,NaN,NaN,1,https://je.jacq.org/JE00017744
65936,Zollinger,Heinrich,NaN,NaN,NaN,NaN,NaN,NaN,2,https://herbarium.bgbm.org/object/B101097046


In [7]:
# check the parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    print(test_collectors.head())


----------------------------------------
show names with **particle** found 534 records:

        family      given suffix  particle  dropping_particle  nick  \
21037    Abreu  Guilherme    NaN        de                NaN   NaN   
4096   Aguilar       M.L.    NaN  Reyna de                NaN   NaN   
60867  Aguilar       M.L.    NaN  Reyna de                NaN   NaN   
16765  Aguilar       M.L.    NaN  Reyna de                NaN   NaN   
46755  Aguilar       M.L.    NaN  Reyna de                NaN   NaN   

      appellation title  occurrenceID_count  \
21037         NaN   NaN                   1   
4096          NaN   NaN                   4   
60867         NaN   NaN                  26   
16765         NaN   NaN                   2   
46755         NaN   NaN                   3   

                                      occurrenceID_first  
21037  http://id.snsb.info/snsb/collection/22086/3086...  
4096        https://herbarium.bgbm.org/object/B100031063  
60867       https://he

Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [8]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)
collectors.tail()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_count,occurrenceID_first
66575,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",5,https://herbarium.bgbm.org/object/B100263330
32851,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",1,https://herbarium.bgbm.org/object/B100263331
66576,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.",3,https://herbarium.bgbm.org/object/B100009590
66577,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",1,https://herbarium.bgbm.org/object/B100530714
66578,Жирова,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Жирова, O.",1,https://herbarium.bgbm.org/object/B100630811


In [9]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_first', lambda x: list(x)[0]) # custom function, to get the first entry
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

collectors_unique

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.",1,https://herbarium.bgbm.org/object/B100699397
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,1,https://herbarium.bgbm.org/object/B101149305
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,https://je.jacq.org/JE00010154
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.",5,https://herbarium.bgbm.org/object/B100217620
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.",1,https://herbarium.bgbm.org/object/B101153811
...,...,...,...,...,...,...,...,...,...,...,...
20844,Żelazny,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelazny, J.",4,https://herbarium.bgbm.org/object/B100344466
20845,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",6,https://herbarium.bgbm.org/object/B100263330
20846,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.",3,https://herbarium.bgbm.org/object/B100009590
20847,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",1,https://herbarium.bgbm.org/object/B100530714


In [10]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536

The ngrams function is used as an analyzer in the text search later.

In [11]:
[wd_matchtest['canonical_string'].at[row] for row in range(3)]

['(-Walraevens), O.H.', '(1835-1906), G.A.F.E.', '(1873-1926), S.S.']

In [12]:
[wd_matchtest_fullnames['canonical_string_fullname'].at[row] for row in range(3)]

['(-Walraevens), O Heylen',
 '(1835-1906), Gustav Adolf Ferdinand Eichler',
 '(1873-1926), Søren Sørensen']

In [13]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


print("Example from name:", ngrams('Klazenga, N.'))
print("Example from collectors_unique:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("Example from match-test:", ngrams(wd_matchtest['canonical_string'].at[1]))
print("Example from match-test:", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
Example from name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N'

Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (BGBM) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [18]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step


def getNearestNeighbour(query, this_vectorizer, this_nbrs_data):
    """Calculate the k-nearest distance for query data using package scikit-learn


    @param query: DataFrame the query data to vectorize and transform
    @param this_vectorizer: the vectorizer of TfidfVectorizer
    @param this_nbrs_data: the data of NearestNeighbors calculations
    @return: (distances, indices) distances and indices
    @rtype (int, int)
    """
    queryTFIDF_ = this_vectorizer.transform(query)
    distances, indices = this_nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


def calculateTFIDFmatchingOfData(query_data, match_data):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @return: DataFrame a data frame of matches with columns 'namematch_collector', 'namematch_wikidata', 'namematch_distance'
    """

    import time
    start = time.time()
    query_data = set(query_data)
    # convert list to set for better performance

    print('Vectorizing data. This may take a while...')
    # vectorize wikidata names
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_vector_data = vectorizer.fit_transform(match_data
        # wd_matchtest['canonical_string']
    )
    nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data)
    duration = time.time() - start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    print('Getting nearest neighbours of %s data ...' % len(query_data))
    distances, indices = getNearestNeighbour(query_data, vectorizer, nbrs_data)
    duration = time.time() - start
    print('Completed after %s s' % duration)

    query_data = list(query_data)  # convert back to list

    print('Finding matches build new data frame ...')
    matches = []
    for i, j in enumerate(indices):
        temp = [query_data[i], match_data.values[j][0], round(distances[i][0], 2)]
        matches.append(temp)

    duration = time.time() - start
    print('Building matches done after %s s' % duration)
    matches = pd.DataFrame(
        matches,
        columns=['namematch_collector', 'namematch_wikidata', 'namematch_distance']
    )

    print('Done')
    return matches


In [21]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'])

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index()

matches

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 2.910360813140869 s
Getting nearest neighbours of 20290 data ...
Completed after 115.77805829048157 s
Finding matches build new data frame ...
Building matches done after 115.93459558486938 s
Done


,index,namematch_collector,namematch_wikidata,namematch_distance
0,20289,"Wolf, W.","Wolf, W.",0.00
1,12119,"Asonganyi, J.N.","Asonganyi, J.N.",0.00
2,12120,"Muller, F.","Muller, F.",0.00
3,4389,"Gjellerup, K.","Gjellerup, K.",0.00
4,12124,"Lobin, W.","Lobin, W.",0.00
...,...,...,...,...
20285,14084,Kyusoz,"Nyushko, T.I.",1.22
20286,1408,"Lescrauwaet, E.","Lescot, M.",1.22
20287,17325,"Avjetijezs, V.","Mekhtijeva, N.A.",1.23
20288,12647,Zulptuwaski,"Załęski, E.",1.24


In [22]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'])

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index()

matches_fullnames


Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.94875168800354 s
Getting nearest neighbours of 559 data ...
Completed after 12.172572374343872 s
Finding matches build new data frame ...
Building matches done after 12.177274703979492 s
Done


,index,namematch_collector,namematch_wikidata,namematch_distance
0,558,"Morton, Friedrich","Morton, Friedrich",0.00
1,107,"Bordère, Henri","Bordère, Henri",0.00
2,344,"Wierzbicki, Antal","Wierzbicki, Antal",0.00
3,109,"Stříbrný, Václav","Stříbrný, Václav",0.00
4,340,"Lackström, Emil Frithiof","Lackström, Emil Frithiof",0.00
...,...,...,...,...
554,413,"Entry, Provisional","Şentürk, Ozan",1.20
555,41,"Rainbird, Coldecott","Colden, Jane",1.21
556,541,"Xinjiang, Inst Bio-Ped-Psamm","Insam, J",1.21
557,336,"Vietnitz, Oelsen-von","Schoellenbach, Carl Christoph Oelhafen von",1.22


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

In [23]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed'
    , right_on='namematch_collector'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,index,namematch_collector,namematch_wikidata,namematch_distance
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.",1,https://herbarium.bgbm.org/object/B100699397,15259,"A. Cano, E.","Cano, Á.",0.64
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,1,https://herbarium.bgbm.org/object/B101149305,20193,Aaiki,"Naiki, A.",0.84
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,https://je.jacq.org/JE00010154,13013,"Aaronsohn, A.","Aaronsohn, A.",0.00
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.",5,https://herbarium.bgbm.org/object/B100217620,2067,"Abaouz, A.","Arbaoui, S.",1.12
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.",1,https://herbarium.bgbm.org/object/B101153811,13056,"Abarca, R.","Abarca, L.",0.57


In [24]:
# append full name matches
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed'
    , right_on='namematch_collector'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,index,namematch_collector,namematch_wikidata,namematch_distance
0,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Abdallah, Raffael",1,https://herbarium.bgbm.org/object/B200125981,317,"Abdallah, Raffael","Issel, Raffaele",1.04
1,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,"Abdul, Kadir Bin",1,https://herbarium.bgbm.org/object/B100184021,323,"Abdul, Kadir Bin","Kadir, Zaid bin A",1.00
2,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,"Abreu, Guilherme",1,http://id.snsb.info/snsb/collection/22086/3086...,324,"Abreu, Guilherme","Rau, Guilherme",0.81
3,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"Adá, García",1,https://herbarium.bgbm.org/object/B100296455,282,"Adá, García","Adá, Ramón García",0.78
4,Aghababyan,Mvon,NaN,NaN,NaN,NaN,NaN,NaN,"Aghababyan, Mvon",96,https://herbarium.bgbm.org/object/B100576238,343,"Aghababyan, Mvon","Aghababyan, Vladislav",0.81


In [25]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,index,namematch_collector,namematch_wikidata,namematch_distance
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,https://je.jacq.org/JE00010154,13013,"Aaronsohn, A.","Aaronsohn, A.",0.0
7,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",2,https://herbarium.bgbm.org/object/B100241637,18685,"Abbe, E.C.","Abbe, E.C.",0.0
11,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.",80,https://herbarium.bgbm.org/object/B100181131,16898,"Abbott, J.R.","Abbott, J.R.",0.0
13,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.",4,http://id.snsb.info/snsb/collection/504820/626...,2106,"Abbott, W.L.","Abbott, W.L.",0.0
19,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.",14,https://herbarium.bgbm.org/object/B100046632,9844,"Abedin, S.","Abedin, S.",0.0


In [26]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

Save the plain name matching results only ...

In [27]:
from datetime import datetime
import os
if not os.path.exists('data'):
    os.makedirs('data')

this_output_file='data/bgbm_collectors_matches_wikidata-botanists_%s.csv' % (
    # "20230705"
    datetime.today().strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_all_matches.to_csv(this_output_file)

print("Wrote plain name matches of collector names into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote plain name matches of collector names into data/bgbm_collectors_matches_wikidata-botanists_20230705.csv (2481 kB)


### Merge and Aggregate Matched Data and WikiData’s

TODO
- merge abbreviated and full name data properly
- check what is renamed for output and aggregating
    - TODO continue 7.8.2023: fix canonical_string	+ canonical_string_fullname when merging perhaps on `wikidata.groupby(['canonical_string_to_use_for_match_abbreviated_or_fullname_something'])`
        - or filter True False on those row values
        - or separate runs: one for full name and one for abbr. name
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- distinguish abbrevited match and full name match
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [40]:
# merge now the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_wikidata', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_wikidata', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [45]:
print("Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
collectors_matches_g1_merged_wikidata[criterion].get([
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_collector
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_collector', 'namematch_wikidata', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'itemLabel', 'wikidata_link'
])

Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_collector,namematch_wikidata,namematch_distance,itemLabel,wikidata_link
10703,2,https://dr.jacq.org/DR049432,Kotschy,"Kotschy, T.",0.43,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10704,37,http://id.snsb.info/snsb/collection/16719/2549...,"Kotschy, K.G.T.","Kotschy, T.",0.72,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10705,310,http://id.snsb.info/snsb/collection/117808/176...,"Kotschy, T.","Kotschy, T.",0.00,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10706,5,https://herbarium.bgbm.org/object/B100526350,"Kotschy, Th","Kotschy, T.",0.50,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10707,1,https://herbarium.bgbm.org/object/B100160086,"Kotschyi, C.G.T.","Kotschy, T.",0.92,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10709,1,http://id.snsb.info/snsb/collection/22980/3175...,"Kotschy, C.G.","Kotschy, C.F.",0.63,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
10710,2494,http://id.snsb.info/snsb/collection/108230/167...,"Kotschy, C.G.T.","Kotschy, C.F.",0.76,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
22109,2,https://je.jacq.org/JE00022436,"Kotschy, Carl Georg Theodor","Kotschy, Theodor",0.68,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
22110,1,https://herbarium.bgbm.org/object/B101113772,"Kotschy, Karl Georg Th","Kotschy, Theodor",0.98,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299


In [58]:
print('Group data by canonical names (abbreviated and full name): '
      '1st multiple related WD items (e.g. Q1232456), 2nd multiple WD item labels')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
    print('Run %s: Group by wiki data’s %s, and aggregate/join item(s) '
          'by “…|…”, add new column items_joined ...' % (i + 1, wd_matching_column))
    wdata_joined_items = wikidata.groupby([wd_matching_column])['item'].apply('|'.join).reset_index()
    wdata_joined_items.rename(columns={'item': 'items_joined'}, inplace=True)

    # print("Done. Show examples of items having multiple matching data «|» … ")
    # criterion = wdata_joined_items['items'].map(lambda x: '|' in x)
    # wdata_joined_items[criterion].head()

    print('Run %s:   Merge all based on namematch_wikidata, add item(s) data ...' % (i + 1))
    collectors_matches_g2 = pd.merge(
        collectors_matches_g1_merged_wikidata, wdata_joined_items,
        left_on='namematch_wikidata', right_on=wd_matching_column
        , suffixes=('__wikidata_merge', '__grp_by_items')
        # append to left-data, right-data only when identical column names occur
    )

    # link wikidata items with canonical name string (pipe separated if more than one)
    print('Run %s:   Group by wiki data’s %s, and aggregate/join itemLabel(s) '
          'by “…|…”, add new column item_labels_joined ...' % (i + 1, wd_matching_column))
    wdata_joined_itemlabels = wikidata.groupby([wd_matching_column])['itemLabel'].apply('|'.join).reset_index()
    wdata_joined_itemlabels.rename(columns={'itemLabel': 'item_labels_joined'}, inplace=True)

    print('Run %s:   Merge all based on namematch_wikidata, add itemLabel(s) data ...' % (i + 1))
    collectors_matches_g3 = pd.merge(
        collectors_matches_g2, wdata_joined_itemlabels,
        left_on='namematch_wikidata', right_on=wd_matching_column
        , suffixes=('__grp_by_items', '__grp_by_item_labels')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_group” ...' % (i + 1))
    collectors_matches_group = collectors_matches_g3 \
        if i == 0 \
        else pd.concat([collectors_matches_group, collectors_matches_g3], ignore_index = True)
    
print('Done')

Group data by canonical names (abbreviated and full name): 1st multiple related items (e.g. Q1232456), 2nd multiple WD item labels
Run 1: Group by wiki data’s canonical_string, and aggregate/join item(s) by “…|…”, add new column items_joined ...
Run 1:   Merge all based on namematch_wikidata, add item(s) data ...
Run 1:   Group by wiki data’s canonical_string, and aggregate/join itemLabel(s) by “…|…”, add new column item_labels_joined ...
Run 1:   Merge all based on namematch_wikidata, add itemLabel(s) data ...
Run 1:   Build data frame “collectors_matches_group” ...
Run 2: Group by wiki data’s canonical_string_fullname, and aggregate/join item(s) by “…|…”, add new column items_joined ...
Run 2:   Merge all based on namematch_wikidata, add item(s) data ...
Run 2:   Group by wiki data’s canonical_string_fullname, and aggregate/join itemLabel(s) by “…|…”, add new column item_labels_joined ...
Run 2:   Merge all based on namematch_wikidata, add itemLabel(s) data ...
Run 2:   Build data fr

In [71]:
print("Show examples of item_labels_joined having multiple matching data «|» … ")
criterion = collectors_matches_group['item_labels_joined'].map(lambda x: '|' in x)

collectors_matches_group[criterion].get([ # empty 
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_collector
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_collector', 'namematch_wikidata', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'item_labels_joined', 'items_joined'
], default="...get: Are data empty or it has probably a wrong named column?")


Show examples of item_labels_joined having multiple matching data «|» … 


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_collector,namematch_wikidata,namematch_distance,item_labels_joined,items_joined
8,1,https://herbarium.bgbm.org/object/B600197578,"Abbasju, M.","Abbas, A.",0.95,Alia Abbas|Abdulla Abbas,http://www.wikidata.org/entity/Q60141229|http:...
9,1,https://herbarium.bgbm.org/object/B600197578,"Abbasju, M.","Abbas, A.",0.95,Alia Abbas|Abdulla Abbas,http://www.wikidata.org/entity/Q60141229|http:...
22,4,https://herbarium.bgbm.org/object/B100005623,"Abe, L.M.","Abe, J.",0.85,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...
23,4,https://herbarium.bgbm.org/object/B100005623,"Abe, L.M.","Abe, J.",0.85,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...
24,1,https://herbarium.bgbm.org/object/B100847062,"Abeinaen, B.","Abe, J.",1.14,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...
...,...,...,...,...,...,...,...
22363,6,https://dr.jacq.org/DR030609,"Müller, Hermann","Müller, Hermann",0.00,Hermann Müller|Hermann Müller,http://www.wikidata.org/entity/Q68738|http://w...
22382,6,https://herbarium.bgbm.org/object/B100241696,"Palmer, Edw","Palmer, Edward",0.74,Edward Palmer|Edward Palmer,http://www.wikidata.org/entity/Q3048655|http:/...
22383,6,https://herbarium.bgbm.org/object/B100241696,"Palmer, Edw","Palmer, Edward",0.74,Edward Palmer|Edward Palmer,http://www.wikidata.org/entity/Q3048655|http:/...
22543,1,https://herbarium.bgbm.org/object/BGT0002482,"Xu, Honggui","Xu, Han",1.00,Han Xu|Han Xu,http://www.wikidata.org/entity/Q21613168|http:...


In [64]:
# check what columns we have and what we would keep for further analysis and what to drop
print(list(collectors_matches_group.columns))
# from merge: _x would means normally from left column, _y means from right column
# in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@#  @'
#  ['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 
#  'appellation', 'title', 'canonical_string_collector_parsed', 
#  'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample', 
#  'index', 'namematch_collector', 'namematch_wikidata', 'namematch_distance', 
#  'item', 'itemLabel', 'surname', 'initials', 'canonical_string__wikidata_merge', 
#  'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 
#  'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 
#  'harv_link', 'ipni_link', 'bionomia_link', 'canonical_string__grp_by_items', 
#  'items_joined', 'canonical_string', 'item_labels_joined', 
#  'canonical_string_fullname__wikidata_merge', 
#  'canonical_string_fullname__grp_by_items']

['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 'appellation', 'title', 'canonical_string_collector_parsed', 'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample', 'index', 'namematch_collector', 'namematch_wikidata', 'namematch_distance', 'item', 'itemLabel', 'surname', 'initials', 'canonical_string__wikidata_merge', 'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link', 'canonical_string__grp_by_items', 'items_joined', 'canonical_string', 'item_labels_joined', 'canonical_string_fullname__wikidata_merge', 'canonical_string_fullname__grp_by_items']


Prepare data to save later on …

In [73]:
# Remove superfluous columns TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# TODO check duplicates
collectors_matches_group_simplified = collectors_matches_group.get(
    ['family', 'given', 'canonical_string_collector_parsed', 
      'namematch_collector', # redundant: 'namematch_collector' == 'canonical_string_collector_parsed'
     'namematch_wikidata', 'namematch_distance', 
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
    'items_joined', 'canonical_string', 'canonical_string_fullname', 'surname', 'initials', 'item_labels_joined'
    ], default="...get: Are data empty or it has probably a wrong named column?"
)
# collectors_matches_group = collectors_matches_g3
collectors_matches_group_simplified.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_group_simplified.drop_duplicates(inplace=True)
collectors_matches_group_simplified.head()

/tmp/ipykernel_3444/3301617966.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.sort_values(
/tmp/ipykernel_3444/3301617966.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.drop_duplicates(inplace=True)


,family,given,canonical_string_collector_parsed,namematch_collector,namematch_wikidata,namematch_distance,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,items_joined,canonical_string,canonical_string_fullname,surname,initials,item_labels_joined
4,Aaronsohn,A.,"Aaronsohn, A.","Aaronsohn, A.","Aaronsohn, A.",0.0,NaN,2795076,0000 0001 0948 8581,30592,...,Q2086130,1876.0,1919.0,NaN,http://www.wikidata.org/entity/Q2086130,"Aaronsohn, A.","Aaronsohn, Aaron",Aaronsohn,A.,Aaron Aaronsohn
10,Abbe,E.C.,"Abbe, E.C.","Abbe, E.C.","Abbe, E.C.",0.0,NaN,101473381,0000 0000 7237 8505,30066,...,Q10274118,1905.0,2000.0,NaN,http://www.wikidata.org/entity/Q10274118,"Abbe, E.C.","Abbe, Ernst Cleveland",Abbe,E.C.,Ernst Cleveland Abbe
14,Abbott,J.R.,"Abbott, J.R.","Abbott, J.R.","Abbott, J.R.",0.0,NaN,NaN,NaN,NaN,...,NaN,1968.0,NaN,NaN,http://www.wikidata.org/entity/Q18982386,"Abbott, J.R.","Abbott, J Richard",Abbott,J.R.,J. Richard Abbott
16,Abbott,W.L.,"Abbott, W.L.","Abbott, W.L.","Abbott, W.L.",0.0,NaN,1545420,0000 0000 3712 5377,27518,...,Q635604,1860.0,1936.0,NaN,http://www.wikidata.org/entity/Q635604,"Abbott, W.L.","Abbott, William Louis",Abbott,W.L.,William Louis Abbott
26,Abedin,S.,"Abedin, S.","Abedin, S.","Abedin, S.",0.0,NaN,5859151837993620520007,NaN,69097,...,NaN,1952.0,NaN,NaN,http://www.wikidata.org/entity/Q16142861,"Abedin, S.","Abedin, Sultanul",Abedin,S.,Sultanul Abedin


In [89]:
this_output_file='data/bgbm_collectors_matches_wikidata_items_group_concat_%s.csv' % (
    # "20230705"
    datetime.today().strftime('%Y%m%d_%H%M') # '%Y%m%d_%H%M'
)

collectors_matches_group.to_csv(this_output_file)

print("Wrote groups of collectors matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote groups of collectors matches into data/bgbm_collectors_matches_wikidata_items_group_concat_20230809_1723.csv (10970 kB)


### Merge Data to Individual WikiData Items

For this merge by namematch_wikidata and focus to get individual WikiData items.

In [80]:
print('Merge simply namematch_wikidata to Wiki data for abbreviated and full names... ')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):

    # join wikidata items to avh collectors matches
    #   avh_matches = pd.merge(avh, matches, left_on='label', right_on='name')
    #   avh_matches_t1 = pd.merge(avh_matches, wikidata, left_on='matched_name', right_on='canonical_string')
    # link counts of wikidata items with same canonical name string
    #   avh_matches_t2 = pd.merge(avh_matches_t1, wd_test, left_on="matched_name", right_on="canonical_string")
    #   avh_matches_t2.rename(columns = {list(avh_matches_t2.columns)[-1]: 'dup_count'}, inplace=True)
    
    print('Run %s:   Merge all (collectors matches) using namematch_wikidata, add wikidata ...' % (i + 1))
    collectors_matches_wd1 = pd.merge(
        collectors_all_matches, wikidata,
        left_on='namematch_wikidata', right_on=wd_matching_column,
        suffixes=('__coll_all_matches', '__wd')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_with_wdata” ...' % (i + 1))
    collectors_matches_with_wdata = collectors_matches_wd1 \
        if i == 0 \
        else pd.concat([collectors_matches_with_wdata, collectors_matches_wd1], ignore_index=True)

print('Done')


Group data by canonical names (abbreviated and full name): 1st multiple related items (e.g. Q1232456), 2nd multiple WD item labels
Run 1:   Merge all (collectors matches) using namematch_wikidata, add wikidata ...
Run 1:   Build data frame “collectors_matches_with_wdata” ...
Run 2:   Merge all (collectors matches) using namematch_wikidata, add wikidata ...
Run 2:   Build data frame “collectors_matches_with_wdata” ...
Done


In [86]:
print(list(collectors_matches_with_wdata.columns))
# echo "${text}" | fold --spaces | sed 's@^@#  @'
#  ['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 
#  'appellation', 'title', 'canonical_string_collector_parsed', 
#  'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample', 
#  'index', 'namematch_collector', 'namematch_wikidata', 'namematch_distance', 
#  'item', 'itemLabel', 'surname', 'initials', 'canonical_string', 
#  'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 
#  'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 
#  'harv_link', 'ipni_link', 'bionomia_link']

['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 'appellation', 'title', 'canonical_string_collector_parsed', 'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample', 'index', 'namematch_collector', 'namematch_wikidata', 'namematch_distance', 'item', 'itemLabel', 'surname', 'initials', 'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link']


In [85]:
collectors_matches_with_wdata.drop_duplicates(inplace=True)
collectors_matches_with_wdata

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,...,Q2086130,1876.0,1919.0,NaN,NaN,http://www.wikidata.org/wiki/Q2086130,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23-1,https://bionomia.net/Q2086130
1,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",2,...,Q10274118,1905.0,2000.0,NaN,NaN,http://www.wikidata.org/wiki/Q10274118,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/26-1,https://bionomia.net/Q10274118
2,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.",80,...,NaN,1968.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q18982386,NaN,NaN,https://www.ipni.org/a/20015671-1,NaN
3,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.",4,...,Q635604,1860.0,1936.0,NaN,NaN,http://www.wikidata.org/wiki/Q635604,NaN,https://kiki.huh.harvard.edu/databases/botanis...,NaN,https://bionomia.net/Q635604
4,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.",14,...,NaN,1952.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q16142861,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/35239-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21823,Vihodzevssky,N.,NaN,NaN,NaN,NaN,NaN,NaN,"Vihodzevssky, N.",35,...,NaN,1905.0,1976.0,NaN,NaN,http://www.wikidata.org/wiki/Q22919692,NaN,NaN,NaN,NaN
21824,Avjetijezs,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Avjetijezs, V.",1,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q36642802,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23229-1,NaN
22554,Ketelhut,Maak,NaN,NaN,NaN,NaN,NaN,NaN,"Ketelhut, Maak",1,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q88833606,NaN,NaN,https://www.ipni.org/a/20039750-1,NaN
21825,Zulptuwaski,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zulptuwaski,1,...,NaN,1863.0,1932.0,NaN,NaN,http://www.wikidata.org/wiki/Q5339943,NaN,NaN,NaN,NaN


Save all columns for further analysis

In [87]:
this_output_file='data/bgbm_collectors_matches_wikidata-botanists_all-columns_%s.csv' % (
    "20230705"
    # datetime.today().strftime('%Y%m%d_%H%M') # '%Y%m%d_%H%M'
)

collectors_matches_with_wdata.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_with_wdata.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items of collector matches into data/bgbm_collectors_matches_wikidata-botanists_all-columns_20230705.csv (8585 kB)


In [88]:
# TODO meaningful?
# remove redundant (duplicate (?or empty?)) columns that in any kind are duplicate data (i.e. that we usually do not need)
# do it by transposing it (https://www.statology.org/pandas-drop-duplicate-columns/)
compact_df_tmp=collectors_matches_with_wdata.transpose().drop_duplicates().transpose()
compact_df_tmp.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
this_output_file='data/bgbm_collectors_matches_wikidata-botanists_all-columns-made-unique_%s.csv' % (
    "20230705"
    # datetime.today().strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

compact_df_tmp.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items (unique columns) of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items (unique columns) of collector matches into data/bgbm_collectors_matches_wikidata-botanists_all-columns-made-unique_20230705.csv (8219 kB)


In [ ]:
# TODO further evaluation or filtering, counting, clean up aso.

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
TODO … | Year of first collection
TODO end_date | Year of last collection
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
name_match_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest